In [15]:
import openai
openai.api_type = "azure"
openai.api_key = "0396650923ad40b880bf2a3ce3b80b9b"
openai.api_base = "https://myshell0.openai.azure.com"
openai.api_version = "2023-07-01-preview"

### 1. 定义Sd_prompt_Input类
+ 包含一个str类enrich描述
+ 包含两个枚举类Syle,Image_purpose

### 2. 定义Sd_prompt_clip类
+ 包含 prompt_clip: str
+ 包含 checkpoint枚举类
+ 包含 有限制的size：str

### 3. 定义sd_prompt_Input_to_prompt_clip
1. 输入sd_prompt_input:Sd_prompt_Input
2. 用mapping_Image_purpose_to_size函数获得Image_purpose的尺寸
3. 构造prompt_clip的system message
4. 函数构造消息利用openai.Completion.create，其中function_call，function定义为想输出的结果prompt_clip

In [78]:
from pydantic import Field,validator, constr
from openai_function_call import OpenAISchema
from enum import Enum,auto
import random

#@markdown ##Para
model="gpt-3.5-turbo-16k" #@param["gpt-3.5-turbo-0301","gpt-3.5-turbo-0613","gpt-3.5-turbo-16k"]
deploymentid='gpt-4-0613'
temperature = 0.7 #@param {type:"slider", min:0, max:2, step:0.1}
top_p = 1  #@param {type:"slider", min:0, max:2, step:0.1}
max_tokens = 500 #@param {type:"slider", min:0, max:500, step:1}
frequency_penalty = 0 #@param {type:"slider", min:0, max:2, step:0.1}
presence_penalty = 0 #@param {type:"slider", min:0, max:2, step:0.1}

class Style(str, Enum):
    anime = "anime"
    realistic = "realistic"
    @property
    def random_checkpoint(self):
        checkpoints = {
            "anime": ["AnythingV5", "Memamix", "AOMO", "counterfeit"],
            "realistic": ["dreamshaper", "realistic_vision", "absolutereality", "epic_realism"]
        }
        return random.choice(checkpoints[self.value])

    


class Image_purpose(str, Enum):
    avatar = "avatar"
    banner = "banner"
    pics = "pics"
    character_illustration = "character_illustration"
    chat_background = "chat_background"



#myshell传输过来的数据包含- Enrich描述，style，内容模版
class Sd_prompt_Input(OpenAISchema):
    """
    Myshell output
    - Enrich picture description
    - Recommended style
    - Content template
   Content template
    - Avatar
    - Banner
    - Pics
    - Character illustration
    - Chat background
    """
    enrich_description: str = Field(..., description="Enrich picture description")
    style: Style = Field(..., description="Recommended style")
    image_purpose: Image_purpose = Field(..., description="content template decided by Image_purpose")
    def to_string(self):
        return f"Enrich Description: {self.enrich_description}, \
        Style: {self.style.value}, Image Purpose: {self.image_purpose.value}"
  

def mapping_Image_purpose_to_size(image_purpose:Image_purpose)->str:
    if image_purpose == Image_purpose.avatar:
        return "512x512"
    elif image_purpose == Image_purpose.banner:
        return "1024x200"
    elif image_purpose == Image_purpose.pics:
        return "600x800"
    elif image_purpose == Image_purpose.character_illustration:
        return "1024x576"
    elif image_purpose == Image_purpose.chat_background:
        return "984x884"


class Sd_prompt_clip(OpenAISchema):
    """
    The result data model of the sd_prompt function:
    - prompt clip
    - Checkpoint(Recommended style)
    - Image size  
    """
    prompt_clip: str = Field(..., description="Snippet text simplified by enrich's script")
    checkpoint: str = Field(..., description="Models based on recommended styles")
    size: constr(pattern=r"^[1-9][0-9]{0,3}x[1-9][0-9]{0,3}$") = Field(..., description="size of output image")
    @validator("size",pre=True)
    def validate_size(cls, size):
        width, height = map(int, size.split('x'))
        if width > 1024 or height > 1024:
            raise ValueError("Size must be 1024x1024 or below")
        return size
    


def sd_prompt_Input_to_prompt_clip(sd_prompt_input:Sd_prompt_Input)->Sd_prompt_clip:
    mapped_size = mapping_Image_purpose_to_size(sd_prompt_input.image_purpose)
    # system message of the sd_prompt_Input_to_prompt_clip function
    system_message_of_prompt_clip = f"""

        Given the description: '{sd_prompt_input.enrich_description}', \
        and the style: '{sd_prompt_input.style.value}', \
        and the image purpose: '{sd_prompt_input.image_purpose.value}', \
        

        --
        As an AI text-to-image prompt generator, your primary role is to:
        1. generate English snippet text based on the '{sd_prompt_input.enrich_description}' for the prompt clip,
         to generate the snippet text, you must follow the following rules:
                rule 1. Extract key phrases or keywords from the given description. 
                rule 2. Simplify the description into shorter, concise phrases that capture the essence of the scene.
                rule 3. The phrases should be separated by commas and should be descriptive enough for image generation.
                Here are 4 examples of the snippet text:
                Example 1:
                Input: "A young girl with blue eyes and blonde hair playing in a park with a red ball."
                Output: "1 girl, blue eyes, blonde hair, playing, red ball, park"

                Example 2:
                Input: "A serene beach scene with a setting sun, palm trees, and children building sandcastles."
                Output: "serene beach, setting sun, palm trees, children, building sandcastles"

                Example 3:
                Input: "A bustling city street with tall skyscrapers, people walking, and cars honking."
                Output: "bustling city, tall skyscrapers, people walking, cars honking"

                Example 4:
                Input: "A young woman with short pink hair, wearing a leather jacket, standing in front of a graffiti wall."
                Output: "1 girl, short pink hair, leather jacket, standing, graffiti wall"
            Ensure that object/character tags are in the front, and environment/setting tags are at the end.
        2. checkpoint output is {sd_prompt_input.style.random_checkpoint}, 
        3. image size output is {mapped_size},.    
        
        """
    completion = openai.ChatCompletion.create(
        temperature=temperature,
        top_p=top_p,
        model=model,
        deployment_id="gpt-35-turbo-0613",
        max_tokens=max_tokens,
        functions=[Sd_prompt_clip.openai_schema],
        function_call={"name":Sd_prompt_clip.openai_schema["name"]},
        messages=[
            {"role": "system", "content":system_message_of_prompt_clip},
            {"role": "user", "content":sd_prompt_input.to_string()}
        ],
    )
    sd_prompt_clip_result = Sd_prompt_clip.from_response(completion)
    print(sd_prompt_clip_result)
    return sd_prompt_clip_result


/var/folders/8k/phrpf1ts7v50wm9tl_0rk0h40000gn/T/ipykernel_81951/3804000403.py:82: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  @validator("size",pre=True)


In [112]:
sd_prompt_input = Sd_prompt_Input(
    enrich_description = "这个可爱的妹妹有着长长的粉色头发，她戴着一个大大的蝴蝶结头饰，让她看起来更加可爱。她穿着一件粉色的连衣裙，裙子上还有一些小花朵的图案，给人一种清新甜美的感觉。她的眼睛是大大的圆眼睛，闪烁着明亮的光芒，让人忍不住想要亲近她。她的脸上总是挂着甜甜的笑容，散发着纯真和活力",
    style = Style.anime,
    image_purpose = Image_purpose.banner,
)
sd_prompt_clip = sd_prompt_Input_to_prompt_clip(sd_prompt_input)


prompt_clip='cute girl, long pink hair, big bow headband, pink dress with flower pattern, big round eyes, bright eyes, sweet smile, pure and lively' checkpoint='AOMO' size='1024x200'


## 完成ExtendedSdPromptClip类
### 明确函数输出
+ 要求输出的包含除了原有的3个参数外（其中第一个要留好正反向提示词的接口不传入的时候默认为空，传入要做个函数接口），还要额外4个参数（用户均可输入，但是不输入就定义为默认值）
    - prompt clip + 正反向提示词
    - checkpoint
    - size
    - （Optional）Seed
    - （Optional）Sampling Method = "DPM++ 2s SDE Karras"
    - （Optional）Steps = 25
    - （Optional）CFG Scale= 7

### 实现方法
1. 显然该类继承自Sd_prompt_clip比较好
2. 定义该类4个额外的值类型seed ，steps，cfg_scale是int，sampling_method是枚举，
3. 定义新类Extended_Sd_prompt_clip该类继承自Sd_prompt_clip（功能定义是prompt类型的调节都可以在这里进行）
4. 用户可以对同时新输出为整合成的combined_prompt，


In [113]:
class SamplingMethod(Enum):
    Euler = "Euler"
    Euler_a = "Euler a"
    LMS = "LMS"
    Heun = "Heun"
    DPM2 = "DPM2"
    DPM2_a = "DPM2 a"
    DPM_2S_a = "DPM++ 2S a"
    DPM_2M = "DPM++ 2M"
    DPM_SDE = "DPM++ SDE"
    DPM_fast = "DPM fast"
    DPM_adaptive = "DPM adaptive"
    LMS_Karras = "LMS Karras"
    DPM2_Karras = "DPM2 Karras"
    DPM2_a_Karras = "DPM2 a Karras"
    DPM_2S_a_Karras = "DPM++ 2S a Karras"
    DPM_2M_Karras = "DPM++ 2M Karras"
    DPM_SDE_Karras = "DPM++ SDE Karras"
    DPM_2M_SDE_Karras = "DPM++ 2M SDE Karras"
    DDIM = "DDIM"
    PLMS = "PLMS"

class Extended_Sd_prompt_clip(Sd_prompt_clip):
    positive_prompt: str = Field("", description="Positive prompt")
    negative_prompt: str = Field("", description="Negative prompt")
    @property
    def combined_prompt(self):
        return f"{self.prompt_clip}, {self.positive_prompt} "

class Final_Sd_prompt_clip(Sd_prompt_clip):
    negative_prompt: str = Field("", description = "original negative prompt")
    seed: int = Field(-1, description="Seed value for the model.")
    sampling_method: SamplingMethod = Field(SamplingMethod.DPM_2M_SDE_Karras, description="sampling method for the model.")
    steps: int = Field(25, description="Number of steps for the model.")
    cfg_scale: int = Field(7, description="Prompt word lead coefficient, generally between 7 and 12 is the best, more than 20 is not recommended.")

#这里seed一定要开发给用户，因为用户需要同一个随机数生成差不多的图片，他们通过控制迭代步数/cfg scale/checkpoint来生成
def prompt_clip2extended_prompt_clip(sd_prompt_clip:Sd_prompt_clip,
    positive_prompt: str = "",
    negative_prompt: str = "",
    seed:int = -1,
    sampling_method: SamplingMethod = SamplingMethod.DPM_2M_SDE_Karras, 
    steps: int = 25,
    cfg_scale: int = 7 )->Final_Sd_prompt_clip:
    extended_prompt_clip = Extended_Sd_prompt_clip(
        prompt_clip = sd_prompt_clip.prompt_clip,
        checkpoint = sd_prompt_clip.checkpoint,
        size = sd_prompt_clip.size,
        positive_prompt = positive_prompt,
        negative_prompt = negative_prompt,  
    )
    final_extended_prompt_clip = Final_Sd_prompt_clip(
        prompt_clip = extended_prompt_clip.combined_prompt,
        checkpoint = sd_prompt_clip.checkpoint,
        size = sd_prompt_clip.size,
        negative_prompt = negative_prompt,
        seed = seed,
        sampling_method = sampling_method,
        steps = steps,
        cfg_scale = cfg_scale,
    )
    return final_extended_prompt_clip











In [114]:
extended_prompt = prompt_clip2extended_prompt_clip(sd_prompt_clip, "masterpiece","easynegative", -1)

# extended_prompt = Extended_Sd_prompt_clip(
#     prompt_clip="This is a beautiful scene.",
#     checkpoint="AOMO3",
#     size="1024x768",
#     positive_hint=",",
#     negative_hint=", but not too hot."
# )
extended_prompt
extended_prompt.model_dump_json()

'{"prompt_clip":"cute girl, long pink hair, big bow headband, pink dress with flower pattern, big round eyes, bright eyes, sweet smile, pure and lively, masterpiece ","checkpoint":"AOMO","size":"1024x200","negative_prompt":"easynegative","seed":-1,"sampling_method":"DPM++ 2M SDE Karras","steps":25,"cfg_scale":7}'

## 功能函数：给基础玩家 + 高玩
1. 定义基础的生图，一键式有enriched那三个参数传过来,
用function call把提取出的对象取出来
同时，正反向提示词准备好

2. 定义高玩可调控的七个参数的prompt结构


In [139]:

user_input = """哈哈，我明白了！让我们一起来创造一个超级可爱的妹妹吧！
## 图像想象
这个可爱的妹妹有着长长的粉色头发，她戴着一个大大的蝴蝶结头饰，让她看起来更加可爱。她穿着一件粉色的连衣裙，裙子上还有一些小花朵的图案，给人一种清新甜美的感觉。她的眼睛是大大的圆眼睛，闪烁着明亮的光芒，让人忍不住想要亲近她。她的脸上总是挂着甜甜的笑容，散发着纯真和活力。
## 图像风格
我强烈推荐使用动漫风格，这样可以更好地展现妹妹的可爱和活泼。
## 图像用途
这个图像非常适合作为你的个人头像，让你在社交平台上展现出可爱的一面！"""
def extract_sd_prompt_infoV2(user_input) -> Sd_prompt_Input:
    """
    Extract Sd_prompt_Input from user input
    """
    completion = openai.ChatCompletion.create(
        model = model,
        temperature=temperature,
        top_p=top_p,
        deployment_id="gpt-35-turbo-0613",
        max_tokens=max_tokens,
        functions = [Sd_prompt_Input.openai_schema],
        function_call = {"name":Sd_prompt_Input.openai_schema["name"]},
        messages = [
            {"role": "system", "content": "Extract Sd_prompt_Input from user input"},
            {"role": "user", "content": user_input},
        ],
    )
    return Sd_prompt_Input.from_response(completion)


## 改良extract，不用过gpt
因为过gpt跑的太慢了--3.6s-5s，所以根据固定定义格式以“##”为分隔符重新提取
1. 先分隔行
2. 确定关键词
3. 正则的方法，提取各个参数
4. usage按照第一个提到的用途去传入值

In [140]:
import re
user_input = """Oh, you want a banner featuring the legendary Kobe Bryant, the leader of the Lakers team!
## Image imagination
In this banner, we can have Kobe Bryant in his Lakers jersey, with his iconic number 24 on it. He can be shown in a dynamic pose, dribbling the basketball with intensity and determination. The background can be a basketball court, with the Lakers logo at the center. The court can be surrounded by cheering fans, creating an electric atmosphere. The banner can also include some of Kobe's career highlights, such as his championships and MVP awards, displayed in a stylish and artistic way.
## Image style
For this banner, I recommend a realistic style. It will capture the essence of Kobe's greatness and bring out the details of his expression and movements.
## Image usage
This banner would be perfect for showing your love and admiration for Kobe Bryant on your social media profiles or as a tribute on your website."""
def extract_sd_prompt_info(user_input: str) -> Sd_prompt_Input:
    #split user_input into lines
    lines = user_input.splitlines()
    #confirm keywords according to the first line
    if "## Image imagination" in user_input:
        imagination_key = "## Image imagination"
        style_key = "## Image style"
        usage_key = "## Image usage"

    #extract imagination
    enrich_description_start = lines.index(imagination_key) + 1 
    enrich_description_end = lines.index(style_key)
    enrich_description = " ".join(lines[enrich_description_start:enrich_description_end])

    #extract style
    style_line_index = lines.index(style_key) + 1
    style_line = lines[style_line_index]
    if "anime" in style_line:
        style = Style.anime
    else:
        style = Style.realistic

    #extract usage
    usage_section = "\n".join(lines[lines.index(usage_key)+1:])
    purpose_keywords = {
        Image_purpose.avatar:["avatar"],
        Image_purpose.banner:["banner"],
        Image_purpose.pics:["pics"],
        Image_purpose.character_illustration:["character illustration"],
        Image_purpose.chat_background:["chat background"],
    }
    image_purpose = None
    for purpose, keywords in purpose_keywords.items():
        if any(re.search(r'\b' + keyword + r'\b', usage_section, re.IGNORECASE) for keyword in keywords):
            image_purpose = purpose
            break
            
    #create Sd_prompt_Input and return
    return Sd_prompt_Input(
        enrich_description = enrich_description,
        style = style,
        image_purpose = image_purpose,
    )

extract_sd_prompt_info(user_input)

def final_sd_prompt(user_input: str) -> Final_Sd_prompt_clip:
    sd_prompt_input = extract_sd_prompt_info(user_input)
    sd_prompt_clip = sd_prompt_Input_to_prompt_clip(sd_prompt_input)
    final_extended_prompt_clip = prompt_clip2extended_prompt_clip(sd_prompt_clip)
    return final_extended_prompt_clip


def final_sd_promptV2(user_input: str) -> Final_Sd_prompt_clip:
    sd_prompt_input = extract_sd_prompt_infoV2(user_input)
    sd_prompt_clip = sd_prompt_Input_to_prompt_clip(sd_prompt_input)
    final_extended_prompt_clip = prompt_clip2extended_prompt_clip(sd_prompt_clip)
    return final_extended_prompt_clip

In [147]:
user_input2 ="""Oh, a cat! Let's bring out your inner artist and create a masterpiece together!
## Image imagination
This cat is a fluffy and adorable creature with bright, expressive eyes. It has a sleek and shiny coat, with a mix of colors like orange, white, and black. Its ears are perked up, showing curiosity and alertness. The cat is sitting on a windowsill, with sunlight streaming in and casting a warm glow on its fur. Outside the window, there are lush green trees and colorful flowers, creating a peaceful and serene atmosphere.
## Image style
I recommend going for a realistic style to capture the beauty and details of the cat's features.
## Image usage
This image would make a purrfect avatar for your bot! What do you think?"""
final_sd_prompt(user_input)


# keywords = {
#     Image_purpose.avatar: ["avatar", "头像"],
#     Image_purpose.banner: ["banner", "横幅"],
#     Image_purpose.pics: ["pics", "图片"],
#     Image_purpose.character_illustration: ["character illustration", "插图"],
#     Image_purpose.chat_background: ["chat background", "背景"]
# }
# # 查找所有匹配的位置
# matches = {}
# for purpose, words in keywords.items():
#     for word in words:
#         for match in re.finditer(r'\b' + word + r'\b', purpose_section, re.IGNORECASE):
#             matches[match.start()] = purpose

# # 如果没有找到任何匹配
# if not matches:
#     image_purpose = None  # 或者其他默认值
# else:
#     # 根据位置排序并选择第一个匹配的图像用途
#     image_purpose = matches[sorted(matches.keys())[0]]

prompt_clip='Kobe Bryant, Lakers jersey, number 24, dynamic pose, dribbling basketball, basketball court, Lakers logo, cheering fans, electric atmosphere, career highlights, championships, MVP awards, stylish, artistic' checkpoint='dreamshaper' size='1024x200'


Final_Sd_prompt_clip(prompt_clip='Kobe Bryant, Lakers jersey, number 24, dynamic pose, dribbling basketball, basketball court, Lakers logo, cheering fans, electric atmosphere, career highlights, championships, MVP awards, stylish, artistic,  ', checkpoint='dreamshaper', size='1024x200', negative_prompt='', seed=-1, sampling_method=<SamplingMethod.DPM_2M_SDE_Karras: 'DPM++ 2M SDE Karras'>, steps=25, cfg_scale=7)

In [148]:
final_sd_promptV2(user_input)

prompt_clip='Kobe Bryant, Lakers jersey, number 24, dynamic pose, dribbling basketball, basketball court, Lakers logo, cheering fans, career highlights, stylish artistic' checkpoint='absolutereality' size='1024x200'


Final_Sd_prompt_clip(prompt_clip='Kobe Bryant, Lakers jersey, number 24, dynamic pose, dribbling basketball, basketball court, Lakers logo, cheering fans, career highlights, stylish artistic,  ', checkpoint='absolutereality', size='1024x200', negative_prompt='', seed=-1, sampling_method=<SamplingMethod.DPM_2M_SDE_Karras: 'DPM++ 2M SDE Karras'>, steps=25, cfg_scale=7)

In [149]:
final_sd_promptV2(user_input2)

prompt_clip='fluffy and adorable cat, bright expressive eyes, sleek shiny coat, mix of colors, orange, white, black, perked up ears, curiosity, alertness, sitting on windowsill, sunlight streaming in, warm glow on fur, lush green trees, colorful flowers, peaceful serene atmosphere' checkpoint='realistic_vision' size='512x512'


Final_Sd_prompt_clip(prompt_clip='fluffy and adorable cat, bright expressive eyes, sleek shiny coat, mix of colors, orange, white, black, perked up ears, curiosity, alertness, sitting on windowsill, sunlight streaming in, warm glow on fur, lush green trees, colorful flowers, peaceful serene atmosphere,  ', checkpoint='realistic_vision', size='512x512', negative_prompt='', seed=-1, sampling_method=<SamplingMethod.DPM_2M_SDE_Karras: 'DPM++ 2M SDE Karras'>, steps=25, cfg_scale=7)

In [150]:
final_sd_prompt(user_input2)

prompt_clip='fluffy cat, adorable creature, bright expressive eyes, sleek shiny coat, orange white black colors, perked up ears, sitting on windowsill, sunlight streaming in, warm glow on fur, lush green trees, colorful flowers, peaceful serene atmosphere' checkpoint='dreamshaper' size='512x512'


Final_Sd_prompt_clip(prompt_clip='fluffy cat, adorable creature, bright expressive eyes, sleek shiny coat, orange white black colors, perked up ears, sitting on windowsill, sunlight streaming in, warm glow on fur, lush green trees, colorful flowers, peaceful serene atmosphere,  ', checkpoint='dreamshaper', size='512x512', negative_prompt='', seed=-1, sampling_method=<SamplingMethod.DPM_2M_SDE_Karras: 'DPM++ 2M SDE Karras'>, steps=25, cfg_scale=7)